In [1]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

from streamlit import columns

# Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [2]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [3]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [4]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [5]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District
1,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District
2,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,-600.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
398,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District
399,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District
400,Virudhunagar,Provision materials to Orphanages,Article,1,204302.0,204302,NaN,-4302.0,Adhiparasakthi elderly aged Home,District
401,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District


In [6]:
district_df.duplicated(subset=["NAME OF THE DISTRICT","REQUESTED ARTICLE","QUANTITY","COMMENTS"]).any()

np.False_

In [7]:
valid_articles(district_df["REQUESTED ARTICLE"])

All articles are valid.


In [8]:
district_df.nunique()

NAME OF THE DISTRICT     63
REQUESTED ARTICLE        85
ITEM TYPE                 3
QUANTITY                 51
COST PER UNIT            86
TOTAL COST              194
ALLOTTED FUNDS           48
EXCESS/SHORTAGE          54
COMMENTS                 65
Beneficiary Type          1
dtype: int64

In [9]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()

(district_req[district_req["REQUESTED ARTICLE"]=="AHUJA Radios"]["QUANTITY"].to_list()[0])

1

In [10]:
groupeddt = district_df.groupby("NAME OF THE DISTRICT").agg({
    "TOTAL COST": "sum",
    "ALLOTTED FUNDS": "sum",
    "EXCESS/SHORTAGE": "sum"
}).reset_index()
groupeddt["Check"] = groupeddt["ALLOTTED FUNDS"] -( groupeddt["TOTAL COST"] + groupeddt["EXCESS/SHORTAGE"])
groupeddt

,NAME OF THE DISTRICT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,Check
0,Ariyalur,190600,190000.0,-600.0,0.0
1,Chengalpattu,193120,165000.0,-28120.0,0.0
2,Chennai Central,165560,200000.0,34440.0,0.0
3,Chennai North,252760,250000.0,-2760.0,0.0
4,Chennai South,176612,175000.0,-1612.0,0.0
...,...,...,...,...,...
58,Vellore,675040,675000.0,-40.0,0.0
59,Vijayawada,48464,40000.0,-8464.0,0.0
60,Villupuram,535195,523000.0,-12195.0,0.0
61,Virudhunagar,404302,400000.0,-4302.0,0.0


In [11]:
district_df["TOTAL COST"] == district_df["COST PER UNIT"]*district_df["QUANTITY"]

0      True
1      True
2      True
3      True
4      True
       ... 
398    True
399    True
400    True
401    True
402    True
Length: 403, dtype: bool

In [12]:
district_df[district_df["REQUESTED ARTICLE"]=="Education Aid"].groupby(["NAME OF THE DISTRICT","TOTAL COST"]).agg({"QUANTITY":"sum"})

QUANTITY
NAME OF THE DISTRICT TOTAL COST          
Chennai Central      15000              1
                     20000              2
Chennai North        15000              1
Chennai South        16000              1
                     22000              1
Cuddalore            80000              4
                     100000             4
Dharmapuri Central   25000              1
                     30000              3
Dharmapuri North     20000              1
Dharmapuri South     50000              1
Dindigul             20000              1
                     40000              4
Kanchipuram          15000              3
Kerala State         50000              1
Madurai              10000              1
Mumbai, Navi, Pune   15000              1
                     20000              2
Nilagiri             15000              1
                     20000              2
Pudhucheri           25000              1
Pudhukottai          10000              1
Ranipet              32365              1
Thanjavur            77000              1
                     81850              1
                     134100             1
                     174900             1
Theni (R.)           10000              1
Thiruppur            15000              2
Vellore              5000               1
                     8000               1
                     50000              5
Virudhunagar         75000              1

In [13]:
district_df["TOTAL COST"].sum()

np.int64(22599053)

### Public

In [14]:
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,0,0,0,Rejected. 15000,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,P 290,Dhanalakshmi K,268977092194,No,"Kesavarayanpettai, Melmaruvathur",6383311057,Sewing Machine ORD / Motor,6300,1,6300,No,Public,Article
293,P 293,Manivasagam Villupuram SS48,999999999999,No,Villupuram SS48,9443224594,Gents Cycle,5500,1,5500,No,Public,Article
294,P 298,Yuvanidhi R,982290535343,No,"Sothupakkam, Melmaruvathur",8056846768,Education Aid,26000,1,26000,No,Public,Aid
295,P 287,Soniya M,810012250885,No,Acharapakkam,7695808755,Girls Cycle,5200,1,5200,No,Public,Article


In [15]:
public_df["REQUESTED ARTICLE"].nunique()

44

In [16]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [17]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,1
1,Agri Cart Weeder,1
2,Aluminium Idli Making Box,1
3,Auto- 3 Wheeler,2
4,Bosch Electrician Kit 13 Re,3
5,Business Aid,33
6,Canon Printer 6030B (USB),1
7,Construction Aid,6
8,Desktop Computer - HVY,1
9,Dictionary English - Tamil,500


In [18]:
public_df[(public_df["REQUESTED ARTICLE"]=="Financial Aid")|(public_df["REQUESTED ARTICLE"]=="Business Aid")|(public_df["REQUESTED ARTICLE"]=="Livelihood Aid")|(public_df["REQUESTED ARTICLE"]=="Construction Aid")|(public_df["REQUESTED ARTICLE"]=="Renovation Aid")|(public_df["REQUESTED ARTICLE"]=="Ex gratia for Deceased")&(public_df["QUANTITY"]!=0)]["Total Value"].sum()

np.int64(968000)

In [19]:
public_df[public_df["REQUESTED ARTICLE"]=="Medical Aid"]["Total Value"].sum()

np.int64(168020)

### Institution

In [20]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
16,I 001,Cheyyur Thasildar Office,Laptop - i5,1,60800,60800,"Thasildar Office, Cheyyur, Chengalpattu",9999999999,No,Institution,Article
17,I 002,Adhiparasakthi Institutions,Laptop,81,34000,2754000,Melmaruvathur,044 27529096,No,Institution,Article
18,I 003,Govt School - MKM,Steel Cupboard,2,12390,24780,Marakanam,9444662668,No,Institution,Article
19,I 004,Govt School - KDPM,S Type Chair,5,2242,11210,Kadapakkam,9442826622,No,Institution,Article
20,I 004,Govt School - KDPM,Office Table 4X2,5,4602,23010,Kadapakkam,9442826622,No,Institution,Article
21,I 005,Govt.School - AVPR,Office Table 4X2,5,4602,23010,"Avanipur, Villupuram",8778403593,No,Institution,Article
22,I 006,Govt.School- NMBR,Steel Cupboard,2,12390,24780,Nolambur,94442375680,No,Institution,Article
23,I 007,Govt.School - CYR,Steel Cupboard,3,12390,37170,Cheiyur,8339068395,No,Institution,Article
24,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,1,60000,60000,Chengalpattu,9865336966,No,Institution,Article
25,I 009,"Fire Station, Acharapakkam",Epson Printer L3250 (Lite),1,15800,15800,"Acharapakkam, Chengalpattu",9444279203,No,Institution,Article


In [21]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [22]:
inst_df["REQUESTED ARTICLE"].nunique()

19

In [23]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,100W Flood Light,1
1,Canon EOS 5500 Camera,1
2,Canon TR 4826 Photocopier,1
3,Ceiling Fan- Ord,35
4,Colour Printer HP Smart Tank 790 All in one,1
5,Desktop Computer,2
6,Epson Printer L3250 (Lite),2
7,Grass Cutter Machine,1
8,"HP Printer 126NW (Heavy, All in 1)",1
9,Laptop,82


### Others

In [24]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
0,A 001,Poongodi,Accident Victim,1,50000,50000,Nil,9629291705,No,Others,Aid
1,A 002,Kannagi Mayazhagu,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 339002257674,Others,Aid
2,A 003,Priyanga Gandhi,Accident Victim,1,8000,8000,Kallakurichi,9999999999,A: 545045897495,Others,Aid
3,A 004,Sivagami,Accident Victim,1,10000,10000,Kallakurichi,9999999999,A: 290218141057,Others,Aid
4,A 005,Rani,Accident Victim,1,7500,7500,Kallakurichi,9999999999,A: 907938031037,Others,Aid
5,A 006,Santhi,Accident Victim,1,5000,5000,Kallakurichi,9999999999,A: 906926701129,Others,Aid
6,A 007,Lakshmi,Accident Victim,1,5000,5000,Kallakurichi,9999999999,A: 794881677154,Others,Aid
7,A 008,Tamilarasi,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 911476547286,Others,Aid
8,A 009,Ramalingam,Accident Victim,1,8000,8000,Kallakurichi,9999999999,A: 877352990338,Others,Aid
9,A 010,Pazhani,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 749232813457,Others,Aid


In [25]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [26]:
oth_df["REQUESTED ARTICLE"].nunique()

2

In [27]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY
0,Accident Victim,10
1,Flood Relief Fund,6


In [28]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,100W Flood Light,1
1,3 Ltr Table Top Wet Grinder,1
2,5 HP Bore well Pump / DOL Starter,2
3,AHUJA Radios,1
4,Accident Victim,10
...,...,...
114,Wet Grinder 3 Ltrs,3
115,Wet Grinder 5 Ltrs,3
116,Wet Grinder Floor Model 2 Ltr,28
117,Wood Saw Machine (P),2


In [29]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
final = pd.concat([district_df,public_df,inst_df]).reset_index(drop=True).groupby(["REQUESTED ARTICLE","Beneficiary Type"], as_index=False)["QUANTITY"].sum()
final

Download progress: 100%
Download progress: 100%
Download progress: 100%


,REQUESTED ARTICLE,Beneficiary Type,QUANTITY
0,100W Flood Light,Institution,1
1,3 Ltr Table Top Wet Grinder,District,1
2,5 HP Bore well Pump / DOL Starter,District,2
3,AHUJA Radios,District,1
4,Accident Victim,Others,10
...,...,...,...
145,Wet Grinder 5 Ltrs,District,3
146,Wet Grinder Floor Model 2 Ltr,District,26
147,Wet Grinder Floor Model 2 Ltr,Public,2
148,Wood Saw Machine (P),District,2


In [30]:
print("District QTY",district_df["QUANTITY"].sum())
print("Public QTY",public_df["QUANTITY"].sum())
print("Inst&Oth QTY",inst_df["QUANTITY"].sum())
print("total qty - merge",final["QUANTITY"].sum())
print("total qty - added",district_df["QUANTITY"].sum() + public_df["QUANTITY"].sum()+inst_df["QUANTITY"].sum())

District QTY 13694
Public QTY 789
Inst&Oth QTY 223
total qty - merge 14706
total qty - added 14706


In [31]:
# Pivot the DataFrame to create columns for each Beneficiary Type and total
pivot_df = final.pivot_table(index="REQUESTED ARTICLE",columns="Beneficiary Type",values="QUANTITY",aggfunc="sum",fill_value=0).reset_index()

# Add missing columns for Beneficiary Types and calculate the Total
pivot_df["District"] = pivot_df.get("District", 0)
pivot_df["Public"] = pivot_df.get("Public", 0)
pivot_df["Institution"] = pivot_df.get("Institution", 0)
pivot_df["Others"] = pivot_df.get("Others", 0)

pivot_df["Total"] = pivot_df[["District", "Public", "Institution", "Others"]].sum(axis=1)
pivot_df
ord_req_id = "1ou21kOkXQpL-hoaJ-11av2m7Kwk5hsif65jVOiFaU2Y"
existing_data = read_file(ord_req_id)

# Ensure required columns exist
if "Ordered Quantity" not in existing_data:
    existing_data["Ordered Quantity"] = 0
if "Remaining Quantity" not in existing_data:
    existing_data["Remaining Quantity"] = existing_data["Total"]

# Merge existing data with the new ordered quantity
updated_df = pivot_df.merge(
            existing_data[["REQUESTED ARTICLE", "Ordered Quantity", "Remaining Quantity"]],
            on="REQUESTED ARTICLE",
            how="left",
            suffixes=("", "_old")
        ).fillna({"Ordered Quantity": 0, "Remaining Quantity": lambda x: x["Total"]})

updated_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Others,Public,Total,Ordered Quantity,Remaining Quantity
0,100W Flood Light,0,1,0,0,1,0,1
1,3 Ltr Table Top Wet Grinder,1,0,0,0,1,1,0
2,5 HP Bore well Pump / DOL Starter,2,0,0,0,2,2,0
3,AHUJA Radios,1,0,0,0,1,1,0
4,Accident Victim,0,0,10,0,10,9,1
...,...,...,...,...,...,...,...,...
114,Wet Grinder 3 Ltrs,1,0,0,2,3,3,0
115,Wet Grinder 5 Ltrs,3,0,0,0,3,3,0
116,Wet Grinder Floor Model 2 Ltr,26,0,0,2,28,28,0
117,Wood Saw Machine (P),2,0,0,0,2,2,0


In [32]:
print("District Total",updated_df["District"].sum())
print("Public Total",updated_df["Public"].sum())
print("Institution&oth Total",updated_df["Institution"].sum())


District Total 13694
Public Total 789
Institution&oth Total 207


In [33]:
updated_df["Total"].sum()

np.int64(14706)

### Inventory

In [34]:
inventory_df = read_file(ord_req_id)
inventory_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Others,Public,Total,Ordered Quantity,Remaining Quantity
0,100W Flood Light,0,1,0,0,1,0,1
1,3 Ltr Table Top Wet Grinder,1,0,0,0,1,1,0
2,5 HP Bore well Pump / DOL Starter,2,0,0,0,2,2,0
3,AHUJA Radios,1,0,0,0,1,1,0
4,Accident Victim,0,0,10,0,10,9,1
...,...,...,...,...,...,...,...,...
115,Wet Grinder 3 Ltrs,1,0,0,2,3,3,0
116,Wet Grinder 5 Ltrs,3,0,0,0,3,3,0
117,Wet Grinder Floor Model 2 Ltr,26,0,0,2,28,28,0
118,Wood Saw Machine (P),2,0,0,0,2,2,0


In [35]:
print("Total after reducing Motors: ",inventory_df["Total"].sum() - inventory_df[inventory_df["REQUESTED ARTICLE"]=="Sewing Motor"]["Total"].sum())
print("Total Sum:", inventory_df["Total"].sum() - inventory_df[inventory_df["REQUESTED ARTICLE"]=="Sewing Motor"]["Total"].sum())
print("Ordered + Remaining Sum:", inventory_df["Ordered Quantity"].sum() + inventory_df["Remaining Quantity"].sum() - inventory_df[inventory_df["REQUESTED ARTICLE"]=="Sewing Motor"]["Total"].sum())

print("**Calculation Check**")
num_mismatches = (inventory_df["Total"] != inventory_df["Ordered Quantity"] + inventory_df["Remaining Quantity"]).sum()
print(f"Number of mismatched rows: {num_mismatches}")

Total after reducing Motors:  14686
Total Sum: 14686
Ordered + Remaining Sum: 14686
**Calculation Check**
Number of mismatched rows: 0


### All records Button

In [36]:
district_df = read_file(master_data_id)
district_df["App. No."]=""
cnt = 1
for names in district_df["NAME OF THE DISTRICT"].unique():
    district_df.loc[district_df["NAME OF THE DISTRICT"] == names, "App. No."] = "D " + str(f"{cnt:03}")
    cnt += 1

df1 = district_df[["App. No.","NAME OF THE DISTRICT","REQUESTED ARTICLE","ITEM TYPE","QUANTITY","COST PER UNIT","TOTAL COST","Beneficiary Type","COMMENTS"]]
df1 = df1.rename(columns={"NAME OF THE DISTRICT":"Name","REQUESTED ARTICLE":"Article Name","COST PER UNIT":"Cost Per Unit","QUANTITY":"Quantity","TOTAL COST":"Total Value","COMMENTS":"Comments"})
df1

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
398,D 062,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,District,Rameswari
399,D 062,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,District,Karuppasamy
400,D 062,Virudhunagar,Provision materials to Orphanages,Article,1,204302.0,204302,District,Adhiparasakthi elderly aged Home
401,D 063,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,District,No


In [37]:
public_df = read_file(public_master_id)[["App. No.","Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
public_df

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,P 001,Rekha Elumalai,Handicapped Hand Tricycle,Article,1,7350,7350,Public,No
1,P 002,G.J.Arulmanimaran,Education Aid,Aid,0,0,0,Public,Rejected. 15000
2,P 003,A.Manikandan,Education Aid,Aid,1,15000,15000,Public,Rs.15000
3,P 004,V.Jayavel,Business Aid,Aid,1,10000,10000,Public,Rs.10000
4,P 005,K.Ganesh,Gents Cycle,Article,1,5500,5500,Public,No
...,...,...,...,...,...,...,...,...,...
292,P 290,Dhanalakshmi K,Sewing Machine ORD / Motor,Article,1,6300,6300,Public,No
293,P 293,Manivasagam Villupuram SS48,Gents Cycle,Article,1,5500,5500,Public,No
294,P 298,Yuvanidhi R,Education Aid,Aid,1,26000,26000,Public,No
295,P 287,Soniya M,Girls Cycle,Article,1,5200,5200,Public,No


In [38]:
inst_df = read_file(inst_data_id)[["App. No.","Institution Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
df3  = inst_df.rename(columns={"Institution Name":"Name"})
df3

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,A 001,Poongodi,Accident Victim,Aid,1,50000,50000,Others,No
1,A 002,Kannagi Mayazhagu,Accident Victim,Aid,1,3000,3000,Others,A: 339002257674
2,A 003,Priyanga Gandhi,Accident Victim,Aid,1,8000,8000,Others,A: 545045897495
3,A 004,Sivagami,Accident Victim,Aid,1,10000,10000,Others,A: 290218141057
4,A 005,Rani,Accident Victim,Aid,1,7500,7500,Others,A: 907938031037
5,A 006,Santhi,Accident Victim,Aid,1,5000,5000,Others,A: 906926701129
6,A 007,Lakshmi,Accident Victim,Aid,1,5000,5000,Others,A: 794881677154
7,A 008,Tamilarasi,Accident Victim,Aid,1,3000,3000,Others,A: 911476547286
8,A 009,Ramalingam,Accident Victim,Aid,1,8000,8000,Others,A: 877352990338
9,A 010,Pazhani,Accident Victim,Aid,1,3000,3000,Others,A: 749232813457


In [39]:
consold = pd.concat([df1,public_df,df3]).reset_index(drop=True)
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
737,I 014,Adhiparasakthi Annai Illam,Desktop Computer,Article,1,45500.0,45500,Institution,No
738,I 014,Adhiparasakthi Annai Illam,Podiam Mike- Digimore,Article,1,2200.0,2200,Institution,No
739,I 014,Adhiparasakthi Annai Illam,Colour Printer HP Smart Tank 790 All in one,Article,1,26000.0,26000,Institution,No
740,I 015,Union Public School,Steel Cupboard,Article,1,12390.0,12390,Institution,No


In [40]:
consold["Quantity"].sum()

np.int64(14706)

In [41]:
district_df

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District,D 001
1,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District,D 001
2,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,NaN,NaN,No,District,D 001
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,-600.0,No,District,D 001
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District,D 002
...,...,...,...,...,...,...,...,...,...,...,...
398,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District,D 062
399,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District,D 062
400,Virudhunagar,Provision materials to Orphanages,Article,1,204302.0,204302,NaN,-4302.0,Adhiparasakthi elderly aged Home,District,D 062
401,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District,D 063


In [42]:
district_df[district_df["TOTAL COST"]==0]

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.


In [43]:
fd = district_df[district_df["ITEM TYPE"]=="Article"]
for i in fd["REQUESTED ARTICLE"].unique():
    if fd[fd["REQUESTED ARTICLE"] == i]["COST PER UNIT"].nunique() == 1:
        print("Good")
    else:
        print(i)


Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Two Wheeler
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Provision materials to Orphanages
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good


In [44]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE", "Quantity": "QUANTITY"})[
    ["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name": "REQUESTED ARTICLE", "Quantity": "QUANTITY"})[
    ["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
final = pd.concat([district_df, public_df, inst_df]).reset_index(drop=True).groupby(
            ["REQUESTED ARTICLE", "Beneficiary Type"], as_index=False).sum()

Download progress: 100%
Download progress: 100%
Download progress: 100%


In [45]:
public_df = read_file(public_master_id)
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Cost Per Unit,Quantity,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,0,0,0,Rejected. 15000,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,P 290,Dhanalakshmi K,268977092194,No,"Kesavarayanpettai, Melmaruvathur",6383311057,Sewing Machine ORD / Motor,6300,1,6300,No,Public,Article
293,P 293,Manivasagam Villupuram SS48,999999999999,No,Villupuram SS48,9443224594,Gents Cycle,5500,1,5500,No,Public,Article
294,P 298,Yuvanidhi R,982290535343,No,"Sothupakkam, Melmaruvathur",8056846768,Education Aid,26000,1,26000,No,Public,Aid
295,P 287,Soniya M,810012250885,No,Acharapakkam,7695808755,Girls Cycle,5200,1,5200,No,Public,Article


In [46]:
public_df.isna().any()

App. No.                  False
Name                       True
Aadhar (Without Space)    False
Handicapped (Yes / No)    False
Address                   False
Mobile                    False
Article Name              False
Cost Per Unit             False
Quantity                  False
Total Value               False
Comments                  False
Beneficiary Type          False
ITEM TYPE                 False
dtype: bool

### Bifurcation of category

In [51]:
ad = articles_df[["Articles", "Category","Sequence List"]].rename(columns={"Articles": "Article Name","Category": "Article Category"})
consold = consold.merge(ad, on="Article Name",how='left')
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments,Category,Article Category,Sequence List
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No,Agri,Agri,6
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No,Computers,Computers,39
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No,Computers,Computers,64
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No,Sewing Machine,Sewing Machine,111
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No,Agri,Agri,9
...,...,...,...,...,...,...,...,...,...,...,...,...
737,I 014,Adhiparasakthi Annai Illam,Desktop Computer,Article,1,45500.0,45500,Institution,No,Computers,Computers,39
738,I 014,Adhiparasakthi Annai Illam,Podiam Mike- Digimore,Article,1,2200.0,2200,Institution,No,Electronics,Electronics,88
739,I 014,Adhiparasakthi Annai Illam,Colour Printer HP Smart Tank 790 All in one,Article,1,26000.0,26000,Institution,No,Computers,Computers,35
740,I 015,Union Public School,Steel Cupboard,Article,1,12390.0,12390,Institution,No,Office,Office,116


In [48]:
# grouped = consold.groupby(["Category","Article Name","Name","Beneficiary Type"],as_index=False).agg({"Quantity": "sum", "Total Value":"sum"})
# # grouped["Category"] = grouped["Category"].mask(grouped["Category"].duplicated(), "")
# grouped.to_excel("/Users/aswathshakthi/PycharmProjects/MNP25/Report/Full Report.xlsx",index=False)
# grouped

In [49]:
# grouped["Quantity"].sum()